In [ ]:
from data.models import Generator, Critic, WeightClipper
hparams = {
    "batch_size": 2048,
    "learning_rate_gen": 0.00005,
    "learning_rate_crit": 0.00005,
    "latent_dim": 100,
    "device": device,
    "epochs": 150,
    "critic_iterations" : 2,
    "betas": (0.0, 0.9),



    #28,28 convolutionwerte passen hier noch nicht
    "dc1_num": 8,
    "dc1_size": 6,
    "dc1_stride": 2,#12
    "dc1p_size": 4,
    "dc1p_stride": 1,
    "dc2_num": 16,
    "dc2_size": 2,
    "dc2_stride": 2,#6
    "dc2p_size": 2,
    "dc2p_stride": 1,
    "dc3_num": 32,
    "dc3_size": 3,
    "dc3_stride": 1,#4
    "dc3p_size": 3,
    "dc3p_stride": 1,
    "dc4_num": 64,
    "dc4_size": 2,
    "dc4_stride": 2,#2
    "dc4p_size": 3,
    "dc4p_stride": 1,
    "dc5_num": 128,
    "dc5_size": 2,
    "dc5_stride": 1,#1

    "dl1": 64,
    "dl2": 64,
    "dl3": 64,


    "gl1": 100,
    "gl2": 300,
    "gl3": 900,
    "gl4": 1600,
    "upscale_to": 40, #should be the sqrt of gl4
    #40x40
    "gc1_num": 1,
    "gc1_size": 4,
    "gc1_stride": 1,
    "gc1_out_channels": 512,#n x 64 x 4 x 4
    "gc1p_size": 4,
    "gc1p_stride": 2, 
    "gc2_num": 1,
    "gc2_size": 4,
    "gc2_stride": 2,
    "gc2_out_channels": 256, #6 + 3 + 1 = 10 -> 32 x 10 x 10
    "gc2p_size": 3,
    "gc2p_stride": 1,
    "gc3_num": 1,
    "gc3_size": 4,#
    "gc3_stride": 2,
    "gc3_out_channels": 128, #18 + 3 + 1 = 22 -> 16 x 22 x 22
    "gc3p_size": 3,
    "gc3p_stride": 1,
    "gc4_num": 1,
    "gc4_size": 4,
    "gc4_stride": 1,
    "gc4_out_channels": 64, #22 + 3  = 25 -> 8 x 25 x 25
    "gc5_size": 4,
    "gc5_stride": 1,
    "gc5_out_channels": 1, #25 + 3  = 28 -> 1 x 28 x 28
    

}
generator = Generator(hparams).to(device)
critic = Critic(hparams).to(device)
clipper = WeightClipper()

In [ ]:
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
import numpy as np
generator = Generator(hparams).to(device)
critic = Critic(hparams).to(device)

def gradient_penalty(critic, real, fake, device="cpu"):
    BATCH_SIZE, C, H, W = real.shape
    alpha = torch.rand((BATCH_SIZE, 1, 1, 1)).repeat(1, C, H, W).to(device)
    interpolated_images = real * alpha + fake * (1 - alpha)
    interpolated_images.requires_grad_(True)
    mixed_scores = critic(interpolated_images)
    gradient = torch.autograd.grad(
        inputs=interpolated_images,
        outputs=mixed_scores,
        grad_outputs=torch.ones_like(mixed_scores),
        create_graph=True,
        retain_graph=True,
    )[0]
    gradient = gradient.view(gradient.shape[0], -1)
    gradient_norm = gradient.norm(2, dim=1)
    gradient_penalty = torch.mean((gradient_norm - 1) ** 2)
    return gradient_penalty


def create_tqdm_bar(iterable, desc):
    return tqdm(enumerate(iterable),total=len(iterable), ncols=150, desc=desc)


def train_model_gp(generator, critic, clipper, train_loader, val_loader, tb_logger, hparams, name="WGAN", lambda_gp = 10):
    epochs = hparams.get("epochs", 30)
    batch_size = torch.tensor(hparams["batch_size"]).to(device)
    optimizer_generator = torch.optim.Adam(generator.parameters(), hparams["learning_rate_gen"], betas=hparams["betas"])
    optimizer_critic = torch.optim.Adam(critic.parameters(), hparams["learning_rate_crit"], betas=hparams["betas"])
    #scheduler_generator = torch.optim.lr_scheduler.StepLR(optimizer_generator, step_size=int(epochs * len(train_loader) / 5), gamma=hparams.get('gamma', 0.9))
    #scheduler_critic = torch.optim.lr_scheduler.StepLR(optimizer_critic, step_size=int(epochs * len(train_loader) / 5), gamma=hparams.get('gamma', 0.9))
    i = 0
    z = 0
    for epoch in range(epochs):
        generator.train()
        critic.train()
        training_loss_g = []
        training_loss_c = []
        validation_loss_g = []
        validation_loss_c = []
        training_loop = create_tqdm_bar(train_loader, desc=f'Training Epoch [{epoch + 1}/{epochs}]')
        for train_iteration,batch in training_loop:
            optimizer_critic.zero_grad()
            optimizer_generator.zero_grad()
            batch = batch.to(device)
            fake_batch = torch.randn(size= (batch.shape[0], hparams["latent_dim"])).to(device)
            fake_images = generator(fake_batch).detach()
            pred_real = torch.squeeze(critic(batch)).to(device) #now shape (batch_size)
            pred_fake = torch.squeeze(critic(fake_images)).to(device)  #should be same shape
            gradient_penalty_ = gradient_penalty(critic, batch, fake_images, device=device)
            loss_c = pred_fake.mean() - pred_real.mean() + lambda_gp * gradient_penalty_
            #save loss without gradient penalty for logging
            loss_c_no_gp = pred_fake.mean() - pred_real.mean()
            loss_c.backward()
            optimizer_critic.step()
            #append loss without gradient penalty to logging
            training_loss_c.append(loss_c_no_gp.item())
            training_loop.set_postfix(curr_train_loss = "{:.8f}".format(np.mean(training_loss_c)),
                                      lr = "{:.8f}".format(optimizer_critic.param_groups[0]['lr']))
            tb_logger.add_scalar(f'Critic_{name}/train_loss', loss_c.item(), epoch * len(train_loader) + train_iteration)
            
            if train_iteration % 7 == 1:
                with torch.no_grad():
                        tb_logger.add_figure("Generator:", plot_generator(generator,fake_batch), epoch * len(train_loader) + train_iteration)
            i += 1
            if i == hparams["critic_iterations"]:
                z += 1
                optimizer_critic.zero_grad()
                optimizer_generator.zero_grad()
                fake_batch = torch.randn(size= (hparams["batch_size"], hparams["latent_dim"])).to(device)
                pred_fake = torch.squeeze(critic(generator(fake_batch))).to(device)
                loss_g = -torch.mean(pred_fake)
                loss_g.backward()
                optimizer_generator.step()
                #scheduler_generator.step()
                training_loss_g.append(loss_g.item())
                tb_logger.add_scalar(f'Generator_{name}/train_loss', loss_g.item(), z)
                i = 0

        generator.eval()
        critic.eval()
        val_loop = create_tqdm_bar(val_loader, desc=f'Validation Epoch [{epoch + 1}/{epochs}]')
        with torch.no_grad():
            for val_iterartion, batch in val_loop:
                batch = batch.to(device)
                fake_batch = torch.randn(size= (hparams["batch_size"], hparams["latent_dim"])).to(device)
                pred_real = torch.squeeze(critic(batch)).to(device) #now shape (batch_size)
                pred_fake = torch.squeeze(critic(generator(fake_batch))).to(device)  #should be same shape
                loss_c = -torch.mean(pred_real) + torch.mean(pred_fake)
                validation_loss_c.append(loss_c.item())
                training_loop.set_postfix(curr_train_loss = "{:.8f}".format(np.mean(training_loss_c)),
                                          lr = "{:.8f}".format(optimizer_critic.param_groups[0]['lr']))
                tb_logger.add_scalar(f'Critic_{name}/val_loss', loss_c.item(), epoch * len(train_loader) + train_iteration)
                loss_g = -torch.mean(pred_fake)
                validation_loss_g.append(loss_g.item())




path = "logs"
num_of_runs = len(os.listdir(path)) if os.path.exists(path) else 0
path = os.path.join(path, f'run_{num_of_runs + 1}')

tb_logger = SummaryWriter(path)

# Train the classifier.
unlabled_train_loader = torch.utils.data.DataLoader(unlabeled_train, batch_size=hparams['batch_size'], shuffle=True)
unlabled_val_loader = torch.utils.data.DataLoader(unlabeled_val, batch_size=hparams['batch_size'], shuffle=False)
train_model_gp(generator, critic, clipper, unlabled_train_loader, unlabled_val_loader, tb_logger, hparams= hparams)

In [ ]:
generator.eval()
x = torch.randn(size= (hparams["batch_size"], hparams["latent_dim"])).to(device)
preds = generator(x).cpu().detach().numpy()
for i in range(64):
    plt.subplot(8,8,i+1)
    plt.axis('off')
    plt.imshow(preds[i][0], cmap='gray', interpolation='none')

plt.tight_layout()